In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from my_utils import Dictionary

n_unique = 10

src_dict = Dictionary(['<EOS>'])
tgt_dict = Dictionary(['<BOS>', '<EOS>'])
for n in range(n_unique):
    src_dict.add_word(str(n))
    tgt_dict.add_word(str(n))

In [3]:
from my_utils.toy_data import invert_seq
train = invert_seq(5000, n_unique=n_unique)
test = invert_seq(100, n_unique=n_unique)

In [4]:
import torch
from my_utils import DataLoader
from torch_models.utils import seq2seq

def numericalize(dataset, src_dict, tgt_dict):
    numericalized = [([src_dict(s) for s in src], [tgt_dict(t) for t in tgt]) for src, tgt in dataset]
    return numericalized

# device = 'cuda:0'
device = torch.device('cpu')
trans_func = seq2seq(device)

train_loader = DataLoader(numericalize(train, src_dict, tgt_dict), batch_size=64, trans_func=trans_func)
test_loader = DataLoader(numericalize(test, src_dict, tgt_dict), batch_size=50, trans_func=trans_func)
print(train_loader)

DataLoader(
	datasize: 5000
	batchsize: 64
	n_batches: 79
	trans_func: seq2seq
	device: cpu
)


In [5]:
from torch_models import AttnSeq2Seq, Seq2Seq

embed_size=64
dropout=0.3
model = Seq2Seq(embed_size=embed_size, hidden_size=embed_size, src_vocab_size=len(src_dict), tgt_vocab_size=len(tgt_dict),
                    src_EOS=src_dict('<EOS>'), tgt_BOS=tgt_dict('<BOS>'), tgt_EOS=tgt_dict('<EOS>'),
                    num_layers=1, bidirectional=True, dropout=dropout, rnn='LSTM')
print(model)

Seq2Seq(
  (encoder): RNNEncoder(
    (embedding): Embedding(12, 64, padding_idx=11)
    (rnn): LSTM(64, 64, dropout=0.3, bidirectional=True)
  )
  (decoder): RNNEncoder(
    (embedding): Embedding(13, 64, padding_idx=12)
    (rnn): LSTM(64, 64, dropout=0.3)
  )
  (generator): MLP(
    (fc_out): Linear(in_features=64, out_features=12, bias=True)
    (dropout): Dropout(p=0.3)
    (criterion): CrossEntropyLoss()
    (activation): Tanh()
  )
)


/Users/linghan/anaconda/lib/python3.5/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [7]:
%%time
from my_utils import Trainer, EvaluatorSeq, EvaluatorLoss
from my_utils.misc.logging import init_logger
from torch.optim import Adam, SGD

init_logger()

optimizer = Adam(model.parameters(), lr=0.001)
# optimizer = SGD(model.parameters(), lr=0.01)
evaluator = EvaluatorSeq(model, test_loader, measure='accuracy')
# evaluator = EvaluatorLoss(model, test_loader)

trainer = Trainer(model, train_loader)
trainer.train_epoch(optimizer, max_epoch=5,
              evaluator=evaluator, score_monitor=None)

[2018-10-25 17:07:04,887 INFO] steps [79/79]	loss: 1.6562838418574273	
[2018-10-25 17:07:05,351 INFO] Evaluator accuracy: 0.3316831683168317	
[2018-10-25 17:07:07,565 INFO] steps [79/79]	loss: 1.2721877445148517	
[2018-10-25 17:07:08,049 INFO] Evaluator accuracy: 0.3886138613861386	
[2018-10-25 17:07:10,196 INFO] steps [79/79]	loss: 0.8586157409450675	
[2018-10-25 17:07:10,705 INFO] Evaluator accuracy: 0.655940594059406	
[2018-10-25 17:07:13,001 INFO] steps [79/79]	loss: 0.51019352900831	
[2018-10-25 17:07:13,548 INFO] Evaluator accuracy: 0.7797029702970297	
[2018-10-25 17:07:16,030 INFO] steps [79/79]	loss: 0.3078711759063262	
[2018-10-25 17:07:16,600 INFO] Evaluator accuracy: 0.844059405940594	


CPU times: user 38.1 s, sys: 455 ms, total: 38.6 s
Wall time: 14.3 s


In [15]:
%%time
model.beam_search(inputs)

CPU times: user 134 ms, sys: 2.04 ms, total: 136 ms
Wall time: 72.7 ms


[[4, 6, 7, 4],
 [5, 5, 5, 9],
 [11, 5, 11, 8, 3],
 [5, 7, 11],
 [8, 6, 3, 2, 11],
 [6, 7, 3],
 [3, 5, 8, 8],
 [11, 9, 9],
 [8, 10, 2, 6, 2],
 [11, 11, 4]]

In [16]:
%%time
model.greedy_predict(inputs)

CPU times: user 24.5 ms, sys: 1.75 ms, total: 26.3 ms
Wall time: 13.6 ms


[tensor([ 4,  6,  7,  4]),
 tensor([ 5,  5,  5,  9]),
 tensor([ 11,   5,  11,   8,   3]),
 tensor([  5,   7,  11]),
 tensor([  8,   6,   3,   2,  11]),
 tensor([ 6,  7,  3]),
 tensor([ 3,  5,  8,  8]),
 tensor([ 11,   9,   9]),
 tensor([  8,  10,   2,   6,   2]),
 tensor([ 11,  11,   4])]

In [11]:
iter(train_loader)
l = 10
inputs, targets = next(train_loader)
inputs = inputs[:l]
targets = targets[:l]
generated = model.predict(inputs)
print('======= input ======')
for seq in inputs:
    print([src_dict[s.item()] for s in seq])
print('======= output ======')
for seq in generated[:l]:
    print([tgt_dict[s] for s in seq])

======= input ======
['2', '5', '4', '2']
['4', '7', '3', '3', '3']
['1', '6', '9', '3', '9']
['9', '5', '3']
['9', '0', '1', '4', '6']
['1', '5', '4']
['3', '3', '6', '1']
['7', '7', '9']
['0', '4', '0', '8', '6']
['2', '9', '9']
======= output ======
['2', '2', '4', '5']
['3', '3', '3', '7']
['9', '3', '9', '6', '1']
['3', '5', '9']
['6', '4', '1', '0', '9']
['4', '5', '1']
['1', '3', '6', '6']
['9', '7', '7']
['6', '8', '0', '4', '0']
['9', '9', '2']
